In [15]:
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import matplotlib
import asciitable
import pyfits
import os
import physical_scale_func as phys
import re
import pyregion
import csv
import random

# Set Up 

In [45]:
PATH = '/Users/Roozbeh/Desktop/Work/paper3/V2/'
SexPar = '/Users/Roozbeh/Desktop/Work/Old_Projects/Codes/sex/True.param'
crit_narrow = [(0.5,1,11.02,11.27),(1,1.5,10.95,11.20),\
                (1.5,2,10.85,11.10),(2,2.5,10.74,10.99)]
#fields = ['uds','gs','cosmos','egs']
fields = ['gn']

# Constructing Postage Stamp 

### Eliminating Flaged Images/Objects 

In [46]:
def preprocess_cats(catalogue_name):
    """
    Opening the catalogue and removing the flagged objects
    
    Input: Catalogue Name    
    Output: a Panda DF
    """
    
    print 'reading the catalogue...'
    data = asciitable.read(PATH + '/SM/CANDELS/' + catalogue_name)
    print 'catalogues is read!'
    
    # Convert the numpy array to Data Frame -> more freedom
    data_DF = pd.DataFrame(data)
    # Removing flagged, AGNs, and Stars
    # This will get rid of only 10% of the sample
    if 'AGNFlag' in data_DF.columns.values:
        conditions = np.array(data_DF.PhotFlag == 0) *\
                     np.array(data_DF.StarFlag < 0.7) *\
                     np.array(data_DF.AGNFlag == 0)
    elif 'StarFlag' in data_DF.columns.values:
        conditions = np.array(data_DF.PhotFlag == 0) *\
                     np.array(data_DF.StarFlag < 0.7)
    elif 'PhotFlag' in data_DF.columns.values:
        conditions = np.array(data_DF.PhotFlag == 0) 
        
    conditions = pd.Series(conditions)
    data_DF = data_DF[conditions]
    return data_DF    

### Constant Cumulative Number Density

In [47]:
def number_density_selection(catDF, n_density_criteria = crit_narrow):
    """
    Creats a list of Objects Sample Candidates Sample ID
    """
    sample_ID = []
    # Input catDF is a Pandas DF so we need to do iterrows()  
    for index, entry in catDF.iterrows():
        photoz = entry['zbest']
        ## In case it's not already log10 value
        if entry['M_neb_med'] > 100:
            mass = pylab.log10(entry['M_neb_med'])  
        else:
            mass = entry['M_neb_med']    
        for crit in n_density_criteria:
            zLow, zHigh, massLow, massHigh = crit[0], crit[1], crit[2], crit[3]
            if zLow <= photoz < zHigh and massLow <= mass < massHigh:
                sample_ID.append(entry)
    return pd.DataFrame(sample_ID)

### Postage Stamps 

In [48]:
def create_postage_stamp(sample_list, Field):
    """
    It creates a drz and wht postage stamp for each object in the sample list
    
    Input: sample list and Field name (e.g. uds)
    Output: postage stamps along with the sample list catalogued! 
    """
    # Turing the FITS files to arrays
    image = pyfits.open(PATH[:-3] + 'CANDELS_FITS/' + Field + '_drz.fits')
    imagewht = pyfits.open(PATH[:-3] + 'CANDELS_FITS/'+ Field + '_wht.fits')
    pixels = image[0].data
    pixelswht = imagewht[0].data    
    
    for index, each in sample_list.iterrows():

        ID = each['Seq']
        ra, dec = each['RAdeg'], each['DECdeg']
        ## In case it's not already log10 value
        if each['M_neb_med'] > 100:
            mass, z = pylab.log10(each['M_neb_med']), each['zbest']
        else:
            mass, z = each['M_neb_med'], each['zbest']        
        ## Creating a ds9 region
        r = open('location.reg', 'w')
        r.writelines('global color=red dashlist=8 3 width=1 font="helvetica 10 normal"\
        select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1'+'\n')
        r.writelines('fk5'+'\n')
        ra = each[1]
        dec = each[2]
        r.writelines('box(' + str(ra) + ',' + str(dec) + ',9",9",0)' + '\n')
        r.close()
    
        # Converting from WCS to Physical        
        r = pyregion.open('location.reg')
        r2 = r.as_imagecoord(image[0].header)
        x = int(r2[0].coord_list[0])
        y = int(r2[0].coord_list[1])
    

        # Setting the postage stamp size in pixels based on the redshift
        if z > 1.5:
            stamp_size = 100
        else:
            stamp_size = 150
    
        # Creating drz image postage stamp
        temp = pixels[y-stamp_size:y+stamp_size,x-stamp_size:x+stamp_size]
        Postage = pyfits.PrimaryHDU(temp)
        hdulist = pyfits.HDUList([Postage])
        hdulist.writeto(PATH + '/Fits/sample/z' + str(round(z,1)) + \
                        '_m' + str(round(mass,1)) + '_' + str(ID) +'_'+ Field + '_drz.fit')

        # Creating wht image postage stamp    
        tempwht = pixelswht[y-stamp_size:y+stamp_size,x-stamp_size:x+stamp_size]
        Postagewht = pyfits.PrimaryHDU(tempwht)
        hdulistwht = pyfits.HDUList([Postagewht])
        hdulistwht.writeto(PATH + '/Fits/sample/z' + str(round(z,1)) + \
                        '_m' + str(round(mass,1)) + '_' + str(ID) +'_'+ Field + '_rms.fits')

        pd.DataFrame.to_csv(sample_list,PATH + '/SM/' + Field + '_sample.cat',index=False)

In [8]:
for field in fields:
    DF = preprocess_cats(field + '.cat')
    sample = number_density_selection(DF)
    print field, len(sample)
    create_postage_stamp(sample, field)

os.system('rm location.reg')

reading the catalogue...
catalogues is read!
gn

/Users/Roozbeh/anaconda/lib/python2.7/site-packages/pyregion/wcs_helper.py:270: PyfitsDeprecationWarning: The ascard function is deprecated as of version 3.1 and may be removed in a future version.

        Use the `.cards` attribute instead.
  if hasattr(header, 'ascard'):
/Users/Roozbeh/anaconda/lib/python2.7/site-packages/pyfits/card.py:70: PyfitsDeprecationWarning: The CardList class has been deprecated; all its former functionality has been subsumed by the Header class, so CardList objects should not be directly created.  See the PyFITS 3.1.0 CHANGELOG for more details.
  PyfitsDeprecationWarning)
/Users/Roozbeh/anaconda/lib/python2.7/site-packages/pyregion/wcs_helper.py:280: PyfitsDeprecationWarning: The ascard function is deprecated as of version 3.1 and may be removed in a future version.

        Use the `.cards` attribute instead.
  header = repr(header.ascard).encode('latin1')


 76


0

# Removing Challenging Galaxies 

In [14]:
imageList = os.listdir(PATH + 'Fits/Sample/')

for image in imageList:
    if re.search('drz', image):
        os.system('ds9 -scale log -zoom 4 ' + PATH + 'Fits/Sample/' + image + ' & ')
        answer = raw_input('chellneging? (ch/ok/fi):   ')
        if answer == 'ch':
            print 'Challenging!'
            os.system('mv ' + PATH + 'Fits/Sample/' + image\
                     + '  ' + PATH + 'Fits/Sample/CHALLENGING/')
            os.system('mv ' + PATH + 'Fits/Sample/' + image[:-7] + 'rms.fits'\
                     + '  ' + PATH + 'Fits/Sample/CHALLENGING/')
        elif answer == 'fi':
            print 'Fine!'
            os.system('mv ' + PATH + 'Fits/Sample/' + image\
                     + '  ' + PATH + 'Fits/Sample/FINE/')
            os.system('mv ' + PATH + 'Fits/Sample/' + image[:-7] + 'rms.fits'\
                     + '  ' + PATH + 'Fits/Sample/FINE/')
        elif answer == 'ok':
            print 'Okay!'
            os.system('mv ' + PATH + 'Fits/Sample/' + image\
                     + '  ' + PATH + 'Fits/Sample/OKAY/')
            os.system('mv ' + PATH + 'Fits/Sample/' + image[:-7] + 'rms.fits'\
                     + '  ' + PATH + 'Fits/Sample/OKAY/')
        else:
            break

chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   ch
Challenging!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   ok
Okay!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   ok
Okay!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   ch
Challenging!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   ok
Okay!
chellneging? (ch/ok/fi):   ok
Okay!
chellneging? (ch/ok/fi):   ok
Okay!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   fi
Fine!
chellneging? (ch/ok/fi):   ok
Okay!
chellneging? (

# Constructing Mask Images

In [49]:
def run_sex(imageName, thresh = 1.2, par = SexPar):
    """
    creates a catalogue of all the objet in the image which
    later will be used for creating mask image and pixel list
    
    input: image name, detection threshold, and sextractor parameter file
    output: catalogue and the segmentation map (for sanity check)
    """
    input = open(par,'r')
    sex_par = input.readlines()
    input.close()

    sex_par[6] = 'CATALOG_NAME     ' + PATH + '1D/' + imageName[:-8] + '.cat  ' + '\n'
    sex_par[17] = 'DETECT_THRESH    ' + str(thresh) + '   ' + '\n'

    output = open(par,'w')
    output.writelines(sex_par)
    output.close()

    os.system("sex -c " + par + " " + PATH + 'Fits/Sample/FINE/' + imageName)
    os.system('mv seg.fits ' " " + PATH + 'Fits/Sample/FINE/' + imageName[:-8] + '_seg.fits')    

In [50]:
def regionFile_for_masking(imageName, factor=1.5):
    """
    Using the catalogue which is created by run_sex, this creates a region file.
    This region file will be used for masking and creating pl.
    It does't mask anything in the central 20 pixels of the image
    """
    #Finding major axis from area and ellipticity
    def major_axis(area,e):
        return np.sqrt(area/(np.pi*(1-e)))         

    cat = asciitable.read(PATH + '1D/' + imageName[:-8] + '.cat')
    region = open(PATH + '1D/' + imageName[:-8] + '.reg','w')
    image_size = len(pyfits.open(PATH + 'Fits/Sample/FINE/' + imageName)[0].data)

    for i in range(len(cat)):
      
        x = cat[i][1]
        y = cat[i][2]
        area = cat[i][3]
        e = cat[i][4]
        PA = cat[i][5]

        # ignores the central 10 pixels
        if x > image_size/2 - 5 and x < image_size/2 + 5 and\
           y > image_size/2 - 5 and y < image_size/2 + 5:
                continue
        else:
            r1 = factor*major_axis(area,e)                              
            r2 = (1-e)*r1
    
            region.write('global color=green dashlist=8 3 width=1 font="helvetica 10 normal"\
            select=1 highlite=1 dash=0 fixed=0 edit=1 move=1 delete=1 include=1 source=1\nimage\n\
            ellipse('+str(x) + ',' + str(y) + ',' + str(r1) + ',' + str(r2)+','+str(PA)+')\n\n')
    region.close()

In [51]:
def mask_image(imageName):
    """
    Using the region file, the mask image and bad pixel list is created
    """

    image_size = len(pyfits.open(PATH + 'Fits/Sample/FINE/' + imageName)[0].data)    
    
    zeros_array = np.zeros([image_size,image_size])
    tempImage = pyfits.PrimaryHDU(zeros_array)
    emptyImage = tempImage.data

    r = pyregion.open(PATH + '1D/' + imageName[:-8] + '.reg')
    rmask = r.get_mask(hdu=tempImage)
    emptyImage[np.where(rmask==True)] = 1000
    
    tempImage.data = emptyImage
    tempImage.writeto(PATH + 'Fits/Sample/FINE/' + imageName[:-8] + '_mm.fits')

    imcopy_con = open('/Users/Roozbeh/Desktop/Work/Old_Projects/Codes/iraf-par/imcopy.par','r')
    imcopy = imcopy_con.readlines()
    imcopy_con.close()

    imcopy[0] = 'imcopy.input = ' + PATH + 'Fits/Sample/FINE/' + imageName[:-8] + '_mm.fits' + '\n'
    imcopy[1] = 'imcopy.output = ' + PATH + 'Fits/Sample/FINE/' + imageName[:-8] + '.pl' + '\n'

    output = open('/Users/Roozbeh/Desktop/Work/Old_Projects/Codes/iraf-par/imcopy.par','w')
    output.writelines(imcopy)
    output.close()

    os.system('/usr/stsci/iraf/irafbin/bin.macintel/x_images.e imcopy\
              @/Users/Roozbeh/Desktop/Work/Old_Projects/Codes/iraf-par/imcopy.par')

In [52]:
def create_cleaned_image(imageName):
    """
    Creates an image where all the bad pixels are replaced with the
    random background pixels.
    This is not necessarily very helpful. 
    """
    image_size = len(pyfits.open(PATH + 'Fits/Sample/FINE/' + imageName)[0].data)
    mm_image =  pyfits.open(PATH + 'Fits/Sample/FINE/' + imageName[:-8] + '_mm.fits')   
    mm_pixels = mm_image[0].data
    
    pixelValues = []
    sky = pyfits.open(PATH + 'Fits/Sample/FINE/' + imageName)
    bkg_pixel = sky[0].data

    for axis1 in range(len(mm_pixels)):
        for axis2 in range(len(mm_pixels[0])):
            if mm_pixels[axis1,axis2] == 0:
                # and not near the center (i.e., galaxy)
                if axis1 not in range(int(image_size/2)-50, int(image_size/2)+50) and\
                   axis2 not in range(int(image_size/2)-50, int(image_size/2)+50):
                        pixelValues.append(bkg_pixel[axis1,axis2])

    median = str(np.median(pixelValues))
    mean = str(np.mean(pixelValues))
    percentiles = str(np.percentile(pixelValues,(10,25,50,75,90)))

    print 'median:'+ median, 'mean:'+mean, 'percentiles:'+percentiles

    for axis1 in range(len(mm_pixels)):
        for axis2 in range(len(mm_pixels[0])):
            if mm_pixels[axis1,axis2] > 0:
                k = random.randrange(len(pixelValues))
                bkg_pixel[axis1,axis2] =  pixelValues[k]

    sky[0].data = bkg_pixel
    sky.writeto(PATH + 'Fits/Sample/FINE/' + imageName[:-8] + '_clean.fits')

In [53]:
imageList = os.listdir(PATH + 'Fits/Sample/FINE/')

for image in imageList:
    if re.search('drz.fit', image):
        print image
        run_sex(image)
        regionFile_for_masking(image)
        mask_image(image)
        create_cleaned_image(image)

z0.6_m11.1_11494_uds_drz.fit
median:-0.000990212 mean:-0.000979673 percentiles:[-0.00634238 -0.0037638  -0.00099021  0.00183897  0.00435953]
z0.6_m11.1_16306_gn_drz.fit
median:0.000304887 mean:0.00033053 percentiles:[-0.00226226 -0.00105417  0.00030489  0.00167913  0.00293738]
z0.6_m11.1_8151_uds_drz.fit
median:-0.000212655 mean:-0.000221155 percentiles:[-0.00552023 -0.00299603 -0.00021265  0.00255906  0.00509287]
z0.6_m11.2_10713_uds_drz.fit
median:-0.000327744 mean:-0.000313271 percentiles:[-0.00560994 -0.00310557 -0.00032774  0.00249715  0.00498833]
z0.7_m11.0_15086_uds_drz.fit
median:-8.74632e-05 mean:-5.81292e-05 percentiles:[ -5.47061861e-03  -2.91039620e-03  -8.74632060e-05   2.75923265e-03
   5.35451178e-03]
z0.7_m11.0_15700.0_cosmos_drz.fit
median:-0.000477723 mean:-0.000557302 percentiles:[-0.00604785 -0.00335071 -0.00047772  0.00234903  0.00483009]
z0.7_m11.0_3002_uds_drz.fit
median:-0.000470099 mean:-0.000451539 percentiles:[-0.00564755 -0.00322357 -0.0004701   0.00223968  

# Building Initial Catalogue 

In [54]:
def findInitial_e_PA_usingSex(imageName):
    """
    finds the measured ellipticity and position angle for the galaxy.
    It is using the fact that the galaxy in at the center of the imgae
    """

    image_size = len(pyfits.open(PATH + 'Fits/Sample/FINE/' + imageName)[0].data)    
    x_galaxy, y_galaxy =  image_size/2, image_size/2   
    
    catSex = asciitable.read(PATH + '1D/' + imageName[:-8] + '.cat')
    for Obj in catSex:
        x = Obj[1]
        y = Obj[2]
        if abs(x-x_galaxy) <= 5 and abs(y-y_galaxy) <= 5:
            e0 = Obj[4]
            PA0 = Obj[5]
    return (e0, PA0)

In [55]:
def ellipse(imageName, e0, PA0, hold_e='no', hold_PA='no', magzpt=26, step_size=0.02):
    """
    Baically does what ELLIPSE/IRAF does but have limited options for editing
    """
    image_size = len(pyfits.open(PATH + 'Fits/Sample/FINE/' + imageName)[0].data)    
    if hold_e == 'no':
        CofG_or_1st = '1st'
    else:
        CofG_or_1st = 'CofG'

    fp = open(PATH + '/Par/ellipse.par','r')
    ellipse_par = fp.readlines()
    fp.close()  

    ellipse_par[0] = 'ellipse.input = ' + PATH  + 'Fits/Sample/FINE/' + imageName + '\n'
    ellipse_par[1] = 'ellipse.output = ' + PATH + '1D/' + imageName[:-8] + '_' + CofG_or_1st + '.bin' + '\n'
    ellipse_par[2] = 'geompar.x0= ' + str(image_size/2) + '\n'
    ellipse_par[3] = 'geompar.y0= ' + str(image_size/2) + '\n'
    ellipse_par[4] = 'geompar.ellip0 = ' + str(e0) + '\n'
    ellipse_par[5] = 'geompar.pa0 = ' + str(PA0) + '\n'
    ellipse_par[6] = 'magpar.mag0 = ' + str(magzpt) + '\n'
    ellipse_par[7] = 'geompar.maxsma = ' + str(image_size/2) + '\n'
    ellipse_par[8] = 'controlpar.hellip = ' + str(hold_e) + '        \n'
    ellipse_par[9] = 'controlpar.hpa = ' + str(hold_PA) + '           \n'
    ellipse_par[10] = 'geompar.step = ' + str(step_size) + '          \n' 
   
    output = open(PATH + '/Par/ellipse' + imageName[:-8] + '.par','w')
    output.writelines(ellipse_par)
    output.close()  
    os.system('/usr/stsci/stsci_iraf-3.14/stsdas/bin.macintel/x_isophote.e \
               ellipse @' + PATH + '/Par/ellipse' + imageName[:-8] + '.par')
    os.system('rm ' + PATH + '/Par/ellipse' + imageName[:-8] + '.par')

In [56]:
def tdump(imageName, CofG_or_1st):
    """
    Baically does what Tdump/IRAF does
    """   
    ff = open(PATH + '/Par/tdump.par','r')
    tdump_param = ff.readlines()
    ff.close()
    
    tdump_param[0] = 'tdump.table = ' +  PATH + '1D/' + imageName[:-8] + '_' + CofG_or_1st + '.bin' + '\n'
    tdump_param[3] = 'tdump.datafile = ' +  PATH + '1D/' + imageName[:-8] + '_' + CofG_or_1st + '.dat' + '\n'
    
    output = open(PATH + '/Par/tdump' + imageName[:-8] + '.par','w')
    output.writelines(tdump_param)
    output.close()
    
    os.system('/usr/stsci/stsci_iraf-3.14/tables/bin.macintel/x_ttools.e \
                tdump @' + PATH + '/Par/tdump' + imageName[:-8] + '.par')  
    os.system('rm ' + PATH + '/Par/tdump' + imageName[:-8] + '.par')

In [57]:
def CofG(imageName):
    """
    Measured Curve of Growth using Ellipse/IRAF
    """
    ### finding initial e & pa and running Ellipse for free e & pa
    e0, pa0 = findInitial_e_PA_usingSex(imageName)
    ellipse(imageName,e0,pa0,'no','no',step_size=0.1)
    tdump(imageName, '1st')
    
    ### Finding the average e & PA for building Curve of Growth
    e_all, PA_all = [], []
    inial_ellipse = asciitable.read(PATH + '1D/' + imageName[:-8] + '_1st.dat')
    for each_ellipse in inial_ellipse:
        ellipticity = each_ellipse[5]
        positionAngle = each_ellipse[7]
        e_all.append(ellipticity)
        # Turn all PA values to positive values
        PA_all.append(positionAngle + (positionAngle<0)*180)

    e_mean = np.mean(e_all)
    # Making sure -90<PA_mean<90
    PA_mean = np.mean(PA_all) - (np.mean(PA_all)>90) * 180

    ### Building the curve of Growth
    ellipse(imageName,e_mean,PA_mean,'yes','yes')
    tdump(imageName, 'CofG')
    return (e_mean, PA_mean)

In [58]:
def SF_or_Queiscent(U,V,J):
    if U-V>1.3 and V-J<1.5 and (U-V) > 0.8*(V-J) + 0.7:
        return 0 #'quiescent'
    else:
        return 1 #'SF'

def round3(x):
    return round(x,3)


def make_float(parameter):
    if type(parameter) != float:
        return float(parameter[0])
    else:
        return parameter

In [59]:
def choice_2col(cat_input, cat_output, par1, par2,PATH=PATH ):
    con = open(PATH + 'SM/CANDELS/' + cat_input,'r')    
    header = con.readline()
    names = header.split()
    par1_index = names.index(par1)
    par2_index = names.index(par2)
    # Extracting the two columns    
    os.system("awk '{print $" + str(par1_index) + ",$" + str(par2_index) + "}' "\
    + PATH + 'SM/CANDELS/' + cat_input + " > " + PATH + 'SM/CANDELS/temp.cat')
    # Editing the header    
    os.system("sed '1s/.*/" + par1 + " " + par2 + "/' " + \
    PATH + 'SM/CANDELS/temp.cat' + " >" + PATH + 'SM/CANDELS/' + cat_output)

##  Putting Everything together 

In [61]:
imageList = os.listdir(PATH + 'Fits/Sample/FINE/')

print 'reaing tables'
gn_sample = asciitable.read(PATH + 'SM/CANDELS/gn_sample.cat')
gs_sample = asciitable.read(PATH + 'SM/CANDELS/gs_sample.cat')
uds_sample = asciitable.read(PATH + 'SM/CANDELS/uds_sample.cat')
cosmos_sample = asciitable.read(PATH + 'SM/CANDELS/cosmos_sample.cat')
egs_sample = asciitable.read(PATH + 'SM/CANDELS/egs_sample.cat')

gn_photo = asciitable.read(PATH + 'SM/CANDELS/gn_photo.cat')
gs_photo = asciitable.read(PATH + 'SM/CANDELS/gs_photo.cat')
uds_photo = asciitable.read(PATH + 'SM/CANDELS/uds_photo.cat')
cosmos_photo = asciitable.read(PATH + 'SM/CANDELS/cosmos_photo.cat')
egs_photo = asciitable.read(PATH + 'SM/CANDELS/egs_photo.cat')

## Creating physical parameters catalogues
choice_2col('gn_phys_all.cat','gn_phys.cat','Seq','SFH_14a')
choice_2col('gs_phys_all.cat','gs_phys.cat','Seq','SFH_14a')
choice_2col('cosmos_phys_all.cat','cosmos_phys.cat','Seq','SFH_14a')
choice_2col('uds_phys_all.cat','uds_phys.cat','Seq','SFH_14a')
choice_2col('egs_phys_all.cat','egs_phys.cat','Seq','SFH_14a')

gn_phys = asciitable.read(PATH + 'SM/CANDELS/gn_phys.cat')
gs_phys = asciitable.read(PATH + 'SM/CANDELS/gs_phys.cat')
uds_phys = asciitable.read(PATH + 'SM/CANDELS/uds_phys.cat')
cosmos_phys = asciitable.read(PATH + 'SM/CANDELS/cosmos_phys.cat')
egs_phys = asciitable.read(PATH + 'SM/CANDELS/egs_phys.cat')

print 'tables are read!'

reaing tables
tables are read!


In [62]:
IC_cat = []

for image in imageList:
    if re.search('drz.fit', image):   
        
        e_1d, PA_1d = CofG(image)  
        CofG_dat = asciitable.read(PATH + '1D/' + image[:-8] + '_CofG.dat')
        mag_1d = CofG_dat[len(CofG_dat)-1][22]
        flux_t = CofG_dat[len(CofG_dat)-1][20]
    
        ## calculating the half light radius based on total flux
        for i in range(len(CofG_dat)):
            if CofG_dat[i][20] > flux_t/2:
                Re_1d = (CofG_dat[i][0] + CofG_dat[i-1][0])/2
                break
                        
        ## Adding the data from candels catalogue
        ID = float(re.search('_\d+',image).group()[1:])
        if re.search('_gn',image):
            candels_data = gn_sample[gn_sample['Seq'] == ID]
            candels_photo = gn_photo[gn_photo['ID_TFIT'] == ID] 
            candels_phys = gn_phys[gn_phys['Seq'] == ID]
        elif re.search('_gs',image):
            candels_data = gs_sample[gs_sample['Seq'] == ID]
            candels_photo = gs_photo[gs_photo['ID_TFIT'] == ID] 
            candels_phys = gs_phys[gs_phys['Seq'] == ID]
        elif re.search('_uds',image):
            candels_data = uds_sample[uds_sample['Seq'] == ID]
            candels_photo = uds_photo[uds_photo['ID_TFIT'] == ID]
            candels_phys = uds_phys[uds_phys['Seq'] == ID]
        elif re.search('_cosmos',image):
            candels_data = cosmos_sample[cosmos_sample['Seq'] == ID]
            candels_photo = cosmos_photo[cosmos_photo['ID_TFIT'] == ID]
            candels_phys = cosmos_phys[cosmos_phys['Seq'] == ID]
        elif re.search('_egs',image):
            candels_data = egs_sample[egs_sample['Seq'] == ID]
            candels_photo = egs_photo[egs_photo['ID_TFIT'] == ID]
            candels_phys = egs_phys[egs_phys['Seq'] == ID]
        
        z = make_float(candels_data['zbest'])
        mass = make_float(candels_data['M_neb_med'])
        if mass > 1000: 
            mass = np.log10(mass)
        U, V, J = make_float(candels_photo['U']), make_float(candels_photo['V']), make_float(candels_photo['J'])
        SF_class = SF_or_Queiscent(U,V,J)
        SFH = int(make_float(candels_phys['SFH_14a']))
        
        IC_cat.append([image[:-4]]+map(round3,[z,mass,U,V,J,Re_1d,mag_1d,e_1d,PA_1d])+[SF_class,SFH])
                                               
        predictions_file = open(PATH + "SM/initial_conditions.csv", "wb")
        open_file_object = csv.writer(predictions_file)
        open_file_object.writerow(["Name",'z','mass','U','V','J',"Re_1d",\
                                    "Mag_1d","e_1d","PA_1d","SF1_Quies0","SFH"])
        open_file_object.writerows(IC_cat)
        predictions_file.close()        